In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tqdm.notebook import tqdm
import sys
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


## Basic Models

Create basic model functions
- __Target model__: A small and simple CNN
- __N Shadow Models__: Same architecture as the target model
- __Attack Models__ : One for every class

In [17]:
def f_target(X_train, y_train, X_test=None, y_test=None, epochs=100):
  """
  Returns a trained target model, if test data are specified we will evaluate the model and print its accuracy
  """
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(32, (3, 3), activation='tanh'))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dense(32, activation='tanh'))

  model.add(layers.Dense(10)   )
  
  optimizer = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  if X_test is None or y_test is None:
    history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_split=0.2)
  else:
    history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_data=(X_test, y_test))
  return model
  
def f_shadow(X_train, y_train, X_test=None, y_test=None, epochs=50):
  # simply return the target model architecture
  return f_target(X_train, y_train, X_test, y_test, epochs)

def __f_attack(X_train, y_train, X_test, y_test, epochs=50):
  print(X_train.shape, X_test.shape)
  model = models.Sequential()
  model.add(layers.Dense(11, activation='relu', input_shape=(X_train.shape[1], )))
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1000, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_data=(X_test, y_test), verbose=True)
  
  return model

def f_attack(X, y):
  # X_i = (class, probability vector, )
  classes = np.unique(train_labels) # all class labels
  with tf.device('/gpu:0'):
  # split to train and test datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.25)
    attack_model = __f_attack(X_train, y_train, X_test, y_test)

  return attack_model


In [3]:
with tf.device('/gpu:0'):
  (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
  train_images = train_images[:2000] # as the paper attack train wiht only 200 records
  train_labels = train_labels[:2000]
  # use the rest as testing - 'out' records
  test_images = np.concatenate((train_images[2000:], test_images))
  test_labels = np.concatenate((train_labels[2000:], test_labels))

170508288/170498071 [==============================] - 3s 0us/step


In [4]:
with tf.device('/gpu:0'):
  target_model = f_target(train_images, train_labels, epochs=50) 

Epoch 1/50
50/50 [==============================] - 31s 12ms/step - loss: 2.0742 - accuracy: 0.2431 - val_loss: 1.9080 - val_accuracy: 0.3025
Epoch 2/50
50/50 [==============================] - 0s 7ms/step - loss: 1.7715 - accuracy: 0.3675 - val_loss: 1.8001 - val_accuracy: 0.3600
Epoch 3/50
50/50 [==============================] - 0s 7ms/step - loss: 1.6692 - accuracy: 0.3963 - val_loss: 1.8167 - val_accuracy: 0.3325
Epoch 4/50
50/50 [==============================] - 0s 7ms/step - loss: 1.5713 - accuracy: 0.4700 - val_loss: 1.7898 - val_accuracy: 0.3725
Epoch 5/50
50/50 [==============================] - 0s 6ms/step - loss: 1.4639 - accuracy: 0.4769 - val_loss: 1.6281 - val_accuracy: 0.4025
Epoch 6/50
50/50 [==============================] - 0s 7ms/step - loss: 1.3469 - accuracy: 0.5456 - val_loss: 1.6780 - val_accuracy: 0.3775
Epoch 7/50
50/50 [==============================] - 0s 7ms/step - loss: 1.2289 - accuracy: 0.5825 - val_loss: 1.5797 - val_accuracy: 0.4425
Epoch 8/50
50/50 [

In [5]:
# return prediction vector
def predict(model, X_i):
  prob_layer = layers.Softmax()
  return prob_layer(model(X_i.reshape((1, 32, 32, 3)))).numpy()[0]

def rand_record(X=None, k=1):
  if X is None:
    # create a whole new record
    X = np.random.randint(0, 255+1, size=32*32*3).reshape((32, 32, 3))
  else:
    X = X.reshape((32*32*3))
    # change k random features
    k_features = np.random.choice(range(X.shape[0]), size=k, replace=False)

    for i in k_features:
      X[i] += np.random.randint(-X[i], 256-X[i]) # subtract/add a number to change the feature 
  
  return X.reshape((32,32,3))

def synthesize(c, target_model, k_min, k_max, conf_min, iter_max, rej_max):
  X = rand_record()
  y_conf_star = 0.0
  j = 0
  k = k_max
  X_star = None

  for iter in range(iter_max):
    y = predict(target_model, X);
    if y[c] >= y_conf_star:
      if y[c] > conf_min and c == np.argmax(y):
        # sample to decide if we return the data
        if np.random.randint(0, 2):
          return X
      y_conf_star = y[c]
      j = 0
      X_star = X
    else:
      # reject and resample X
      j += 1

      if j > rej_max:
        k = max(k_min, math.ceil(k/2))
        j = 0
    
    X = rand_record(X_star, k)
  
  return X_star # failed. return the last successfull record

In [8]:

def divide_dataset(n_shadows, shadow_dataset_size, X, y):
  D_shadows = []
  rng = np.random.default_rng()
  for i in range(n_shadows):
    sample_i = np.random.choice(range(X.shape[0]), shadow_dataset_size, replace=False)
    assert np.unique(sample_i).shape[0] == shadow_dataset_size # sanity check
    D_shadows.append((X[sample_i, :], y[sample_i, :]))
  return D_shadows

# returns a list of 'n_shadows' datasets
def generate_shadow_dataset(target_model, n_shadows, shadow_dataset_size, n_classes, X_test=None, y_test=None):
  # in case we give test data we will just divide those to train the shadow models
  if X_test is not None and y_test is not None:
    return divide_dataset(n_shadows, shadow_dataset_size, X_test, y_test)
  
  
  # helper function to return a datapoint (for sure)
  def get_shadow_datapoint(c):
    X_i = None
    with tf.device('/gpu:0'):
      while X_i is None:
        X_i = synthesize(c, target_model, 1, 32*32*3, 0.65, 100, 5)
    return X_i
  
  D_shadows = []
  for i in range(n_shadows):
    print(f"Generating D_shadow_{i}")
    # uniformly generate X's for all of the classes (y's)
    X_shadow = np.asarray(
        [get_shadow_datapoint(i%n_classes) for i in range(shadow_dataset_size)]
    )
    y_shadow = np.asarray(
        [(i%n_classes) for i in range(shadow_dataset_size)]
    ).reshape((-1, 1))

    D_shadows.append((X_shadow, y_shadow))
  
  return D_shadows

def create_shadows(D_shadows):
  shadow_models = [] # shadow model list

  for D_shadow in D_shadows:
    # sample data to feed/evaluate the model
    X_shadow, y_shadow = D_shadow
    shadow_X_train, shadow_X_test, shadow_y_train, shadow_y_test = train_test_split(X_shadow, y_shadow, shuffle=True, test_size=0.2)

    # generate the shadow model
    shadow_model = f_shadow(shadow_X_train, shadow_y_train, shadow_X_test, shadow_y_test, 50)

    D_shadow = (shadow_X_train, shadow_y_train), (shadow_X_test, shadow_y_test)
    shadow_models.append((shadow_model, D_shadow))

  return shadow_models # return a list where every item is (model, acc), train-data, test-data

In [28]:
# generate shadow datasets
D_shadows = generate_shadow_dataset(target_model, 20, 2500, 10, test_images, test_labels)

In [29]:
# train the shadow models
shadow_models = create_shadows(D_shadows)

Epoch 1/50
63/63 [==============================] - 1s 12ms/step - loss: 2.1346 - accuracy: 0.2285 - val_loss: 1.9941 - val_accuracy: 0.2880
Epoch 2/50
63/63 [==============================] - 0s 7ms/step - loss: 1.8140 - accuracy: 0.3495 - val_loss: 1.8094 - val_accuracy: 0.3580
Epoch 3/50
63/63 [==============================] - 0s 7ms/step - loss: 1.6543 - accuracy: 0.4090 - val_loss: 1.7904 - val_accuracy: 0.3400
Epoch 4/50
63/63 [==============================] - 0s 7ms/step - loss: 1.5356 - accuracy: 0.4620 - val_loss: 1.7306 - val_accuracy: 0.3900
Epoch 5/50
63/63 [==============================] - 0s 7ms/step - loss: 1.4383 - accuracy: 0.4985 - val_loss: 1.6635 - val_accuracy: 0.4180
Epoch 6/50
63/63 [==============================] - 0s 7ms/step - loss: 1.3222 - accuracy: 0.5600 - val_loss: 1.7609 - val_accuracy: 0.4300
Epoch 7/50
63/63 [==============================] - 0s 7ms/step - loss: 1.2252 - accuracy: 0.5905 - val_loss: 1.7813 - val_accuracy: 0.3720
Epoch 8/50
63/63 [=

In [30]:
# helper function to prepare each shadow dataset batch
def prepare_batch(shadow_model, X, y, in_D):
  #decide membership
  y_member = np.ones(shape=(y.shape[0], 1)) if in_D else np.zeros(shape=(y.shape[0], 1))
  
  # get probability vector
  prob_layer = layers.Softmax() # probability layer implementing softmax for mapping NN results to probabilities in [0, 1]
  prob_vec = prob_layer(shadow_model(X)).numpy()
  
  # return an instance <actual class, prob_vec from shadow model, 'in'/'out' D_target membership> 
  return np.concatenate((y.reshape(-1, 1), prob_vec, y_member), axis=1)

def generate_attack_dataset(shadow_models, n_classes):
  # input is a list where items are model, (X_train, y_train), (X_test, y_test)

  D_attack = None
  # D_attack_i format = <class, prob_vec, membership label (1 or 0)> 
  for shadow_model, ((X_train, y_train), (X_test, y_test)) in shadow_models:
    s = min(X_train.shape[0], X_test.shape[0])
    batch = np.concatenate((
        prepare_batch(shadow_model, X_train[:s], y_train[:s], True), # members of shadow dataset 
        prepare_batch(shadow_model, X_test[:s], y_test[:s], False)   # non members of shadow dataset
    ))   

    D_attack = np.concatenate((D_attack, batch)) if D_attack is not None else batch  

  return D_attack 

In [31]:
D_attack = generate_attack_dataset(shadow_models, 10)

In [32]:
attack_model_bundle = f_attack(D_attack[:, :-1], D_attack[:, -1])

(15000, 11) (5000, 11)
Epoch 1/50
469/469 [==============================] - 3s 6ms/step - loss: 0.5204 - accuracy: 0.7342 - val_loss: 0.3804 - val_accuracy: 0.8558
Epoch 2/50
469/469 [==============================] - 3s 6ms/step - loss: 0.3307 - accuracy: 0.8662 - val_loss: 0.3024 - val_accuracy: 0.8868
Epoch 3/50
469/469 [==============================] - 3s 6ms/step - loss: 0.2958 - accuracy: 0.8837 - val_loss: 0.3257 - val_accuracy: 0.8516
Epoch 4/50
469/469 [==============================] - 3s 5ms/step - loss: 0.2840 - accuracy: 0.8886 - val_loss: 0.3022 - val_accuracy: 0.8932
Epoch 5/50
469/469 [==============================] - 3s 5ms/step - loss: 0.2764 - accuracy: 0.8936 - val_loss: 0.3331 - val_accuracy: 0.8742
Epoch 6/50
469/469 [==============================] - 2s 5ms/step - loss: 0.2728 - accuracy: 0.8946 - val_loss: 0.2652 - val_accuracy: 0.9078
Epoch 7/50
469/469 [==============================] - 3s 6ms/step - loss: 0.2717 - accuracy: 0.8947 - val_loss: 0.2689 - val_

In [33]:
def evaluate_attack(attack_model, X_attack, y_attack, n_classes):
  acc_per_class = []
  for c in range(n_classes):
    class_instances = X_attack[:, 0] == c # get same class samples
    test_loss, test_acc = attack_model.evaluate(X_attack[class_instances, :], y_attack[class_instances], verbose=0)
    acc_per_class.append(test_acc)
    print(f"class-{c+1}: {test_acc}")
  return acc_per_class



In [34]:
# create a test dataset 

D_out = prepare_batch(target_model, test_images, test_labels, False)
D_in = prepare_batch(target_model, train_images, train_labels, True)
print("Testing with 'in' data only:")
res_in = evaluate_attack(attack_model_bundle, D_in[:, :-1], D_in[:, -1], 10)

print("\nTesting with 'out' data only:")
res_out = evaluate_attack(attack_model_bundle, D_out[:2000, :-1], D_out[:2000, -1], 10)

print("\nTesting with all prev data: ")
res_all = evaluate_attack(attack_model_bundle, np.concatenate((D_out[:2000, :-1], D_in[:, :-1])), np.concatenate((D_out[:2000, -1], D_in[:, -1])), 10)


Testing with 'in' data only:
class-1: 0.8316831588745117
class-2: 0.8586387634277344
class-3: 0.7931034564971924
class-4: 0.7794871926307678
class-5: 0.7616822719573975
class-6: 0.7868852615356445
class-7: 0.8550724387168884
class-8: 0.8341708779335022
class-9: 0.7733989953994751
class-10: 0.8325123190879822

Testing with 'out' data only:
class-1: 0.7806122303009033
class-2: 0.8585858345031738
class-3: 0.9333333373069763
class-4: 0.9648241400718689
class-5: 0.9595959782600403
class-6: 0.929729700088501
class-7: 0.7824074029922485
class-8: 0.8238341808319092
class-9: 0.8709677457809448
class-10: 0.8472906351089478

Testing with all prev data: 
class-1: 0.8065326809883118
class-2: 0.8586118221282959
class-3: 0.8618090748786926
class-4: 0.8730964660644531
class-5: 0.856796145439148
class-6: 0.8586956262588501
class-7: 0.817966878414154
class-8: 0.829081654548645
class-9: 0.8238095045089722
class-10: 0.8399015069007874
